# Using Colab an AI agent to my database

Chinnok.db

---




1. List the top 5 customers by total purchase amount.


2. Find the most popular genre in terms of total tracks sold.


3. 4.Retrieve all employees who are managers along with their subordinates.


5. For each artist, find their most sold album.


6. Write a query to get monthly sales trends in the year 2013.


---




In [ ]:
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect("chinook.db")

# List all tables
tables_df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables_df)


               name
0            albums
1   sqlite_sequence
2           artists
3         customers
4         employees
5            genres
6          invoices
7     invoice_items
8       media_types
9         playlists
10   playlist_track
11           tracks
12     sqlite_stat1


In [ ]:
top_5_customers = pd.read_sql_query("""
SELECT c.FirstName, c.LastName, SUM(i.Total) AS TotalPurchaseAmount
FROM customers c
JOIN invoices i
ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalPurchaseAmount DESC
LIMIT 5""", conn)
print(top_5_customers)

  FirstName    LastName  TotalPurchaseAmount
0    Helena        Holý                49.62
1   Richard  Cunningham                47.62
2      Luis       Rojas                46.62
3  Ladislav      Kovács                45.62
4      Hugh    O'Reilly                45.62


In [ ]:
# Query to find the most popular genre based on total tracks sold
query = """
SELECT g.Name, SUM(il.Quantity) AS TotalTracksSold
FROM genres g
JOIN tracks t ON g.GenreId = t.GenreId
JOIN invoice_items il ON t.TrackId = il.TrackId
GROUP BY g.Name
ORDER BY TotalTracksSold DESC
LIMIT 1;
"""

# Execute the query and store the result in a Pandas DataFrame
popular_genre = pd.read_sql_query(query, conn)

# Print the result
print(popular_genre)




   Name  TotalTracksSold
0  Rock              835


In [ ]:
# Query to retrieve managers and their subordinates
query = """
SELECT e1.EmployeeId, e1.FirstName AS ManagerFirstName, e1.LastName AS ManagerLastName,
       e2.EmployeeId AS SubordinateId, e2.FirstName AS SubordinateFirstName, e2.LastName AS SubordinateLastName
FROM employees e1
JOIN employees e2 ON e1.EmployeeId = e2.ReportsTo;
"""

# Execute the query and store the result in a Pandas DataFrame
managers_subordinates = pd.read_sql_query(query, conn)

# Print the result
managers_subordinates




,EmployeeId,ManagerFirstName,ManagerLastName,SubordinateId,SubordinateFirstName,SubordinateLastName
0,1,Andrew,Adams,2,Nancy,Edwards
1,2,Nancy,Edwards,3,Jane,Peacock
2,2,Nancy,Edwards,4,Margaret,Park
3,2,Nancy,Edwards,5,Steve,Johnson
4,1,Andrew,Adams,6,Michael,Mitchell
5,6,Michael,Mitchell,7,Robert,King
6,6,Michael,Mitchell,8,Laura,Callahan


In [ ]:
#For each artist, find their most sold album.

query ="""
SELECT ar.Name AS ArtistName, al.Title AS AlbumTitle
FROM artists ar
JOIN albums al ON ar.ArtistId = al.ArtistId
JOIN tracks t ON al.AlbumId = t.AlbumId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
GROUP BY ar.ArtistId, al.AlbumId
HAVING SUM(ii.Quantity) = (SELECT MAX(TotalQuantity)
                           FROM (SELECT ar2.ArtistId, al2.AlbumId, SUM(ii2.Quantity) AS TotalQuantity
                                 FROM artists ar2
                                 JOIN albums al2 ON ar2.ArtistId = al2.ArtistId
                                 JOIN tracks t2 ON al2.AlbumId = t2.AlbumId
                                 JOIN invoice_items ii2 ON t2.TrackId = ii2.TrackId
                                 WHERE ar2.ArtistId = ar.ArtistId
                                 GROUP BY ar2.ArtistId, al2.AlbumId) AS subquery);"""


most_sold_b_artist = pd.read_sql_query(query, conn)
most_sold_b_artist

,ArtistName,AlbumTitle
0,AC/DC,For Those About To Rock We Salute You
1,Accept,Restless and Wild
2,Aerosmith,Big Ones
3,Alanis Morissette,Jagged Little Pill
4,Alice In Chains,Facelift
...,...,...
171,Martin Roscoe,"Szymanowski: Piano Works, Vol. 1"
172,Göteborgs Symfoniker & Neeme Järvi,Nielsen: The Six Symphonies
173,Michele Campanella,Liszt - 12 Études D'Execution Transcendante
174,Eugene Ormandy,Respighi:Pines of Rome


In [ ]:
# Query to get monthly sales trends in 2013
query = """
SELECT
    strftime('%Y-%m', InvoiceDate) AS sales_month,
    SUM(Total) AS total_sales
FROM invoices
WHERE strftime('%Y', InvoiceDate) = '2013'
GROUP BY sales_month
ORDER BY sales_month;
"""

monthly_sales_2013 = pd.read_sql_query(query, conn)

conn.close()
monthly_sales_2013

,sales_month,total_sales
0,2013-01,37.62
1,2013-02,27.72
2,2013-03,37.62
3,2013-04,33.66
4,2013-05,37.62
5,2013-06,37.62
6,2013-07,37.62
7,2013-08,37.62
8,2013-09,37.62
9,2013-10,37.62


In [ ]:
sol = """
/* Top 5 customers by total purchase amount */

SELECT c.FirstName, c.LastName, SUM(i.Total) AS TotalPurchaseAmount
FROM customers c
JOIN invoices i
ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalPurchaseAmount DESC
LIMIT 5;

/* Most popular genre based on total tracks sold */

SELECT g.Name, SUM(il.Quantity) AS TotalTracksSold
FROM genres g
JOIN tracks t ON g.GenreId = t.GenreId
JOIN invoice_items il ON t.TrackId = il.TrackId
GROUP BY g.Name
ORDER BY TotalTracksSold DESC
LIMIT 1;

/* Managers and their subordinates */

SELECT e1.EmployeeId, e1.FirstName AS ManagerFirstName, e1.LastName AS ManagerLastName,
       e2.EmployeeId AS SubordinateId, e2.FirstName AS SubordinateFirstName, e2.LastName AS SubordinateLastName
FROM employees e1
JOIN employees e2 ON e1.EmployeeId = e2.ReportsTo;

/* Artist with the most sold album */

SELECT ar.Name AS ArtistName, al.Title AS AlbumTitle
FROM artists ar
JOIN albums al ON ar.ArtistId = al.ArtistId
JOIN tracks t ON al.AlbumId = t.AlbumId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
GROUP BY ar.ArtistId, al.AlbumId
HAVING SUM(ii.Quantity) = (SELECT MAX(TotalQuantity)
                           FROM (SELECT ar2.ArtistId, al2.AlbumId, SUM(ii2.Quantity) AS TotalQuantity
                                 FROM artists ar2
                                 JOIN albums al2 ON ar2.ArtistId = al2.ArtistId
                                 JOIN tracks t2 ON al2.AlbumId = t2.AlbumId
                                 JOIN invoice_items ii2 ON t2.TrackId = ii2.TrackId
                                 WHERE ar2.ArtistId = ar.ArtistId
                                 GROUP BY ar2.ArtistId, al2.AlbumId) AS subquery);
/*  monthly sales trends in 2013 */
SELECT
    strftime('%Y-%m', InvoiceDate) AS sales_month,
    SUM(Total) AS total_sales
FROM invoices
WHERE strftime('%Y', InvoiceDate) = '2013'
GROUP BY sales_month
ORDER BY sales_month;
"""

with open("chinnok_solution.sql", "w") as file:
    file.write(sol)
